## Download and extract a 10k/10q XBRL .zip file from the SECs website

### Import packages

In [1]:
# package to create directories
import os
# package used to delete zip files
import shutil
# package for requests
import requests
# libary to open URLs
import urllib.request
# package to work with data frames and store data in .csv files
import pandas as pd
# package to initiate breaks
import time
# package to randomize the length of the breaks
from random import uniform
# package to get the year and the number of the week
from datetime import datetime
# import package to work with regular expressions
import re
# package to work with .zip files
import zipfile

### Specify user-agent and directory to save files

The layout should be "Sample Company Name AdminContact@<sample company domain>.com"
https://www.sec.gov/os/webmaster-faq#developers

In [2]:
company_name = ADD_THE_NAME_HERE_AS_STRING
contact_mail = ADD_THE_MAIL_HERE_AS_STRING

user_agent = "%s %s"%(company_name,contact_mail)
###
directory = ADD_DIRECTORY_HERE_AS_STRING

### Initiate the class SEC Database to download the files

1. When initiating the class, the user-agent to download the files as well as the output directory are stored.
2. On a weekly basis, a list with a firm "cik", "name", "ticker" and "exchange" is downloaded. Old files are deleted.
3. On a weekly basis, a list with all fillings, including their "accessionNumber", "filingDate", "reportDate" and XBRL information is downloaded. Old files are deleted.
4. Download all XBRL fillings of a firm as a .zip and extract the files.

In [3]:
class SEC_DB: 
    ############################################################################################################################
    # 1.
    # When the class is initiated, create the directory to save the files
    def init(self, directory, header):
        # create the directory if it doesn't exist        
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # define the Data Base variable (DB)
        self.DB = directory
        
        # define header
        self.hdr = header
        
    ############################################################################################################################    
    # 2.
    # create function which returns the number of the week in the current year
    def todays_week(self):  
        calender = datetime.today().isocalendar()
        return '%s-%s'%(calender[0], calender[1])
    
    # check if a download is necessary
    def csv_file_exist(self, directory, file_name):
        # check if file exists:
        boolean = os.path.isfile('%s\%s_%s.csv'%(directory, file_name, self.todays_week()))
        return boolean
    
    # check if there are older files to delete
    def older_csv_file_exist(self, directory, file_name):
        # check if older file exists, with the same prefix but another suffix
        for fname in os.listdir('%s'%(directory)):
            if ((fname.startswith('%s'%(file_name)) == True) and
                (fname.endswith('%s.csv'%(self.todays_week())) == False)):
                # if older file exists, delete it
                os.remove('%s\%s'%(directory, fname))    
    
    # define function, to download master company key file, 
    # if the newest is not already on disk     
    def edgar_companies(self, file_name='EdgarCompanies'):
        save_path = self.DB
        # check if it already exists
        if self.csv_file_exist(self.DB, file_name) != True:
            
            time.sleep(uniform(1, 2)) # break between 1 - 2 seconds            

            # link to the overview of companies (no funds) in the edgar database
            URL = 'https://www.sec.gov/files/company_tickers_exchange.json'
            # save data to JSON
            r = requests.get(URL, headers=self.hdr)
            data = r.json()

            # extract the features
            # headers are stored in the "fields" section and values in the "item" section
            df = pd.DataFrame([{data['fields'][0]: item[0], data['fields'][1]: item[1],
                                data['fields'][2]: item[2], data['fields'][3]: item[3],} 
                               for item in data['data'] ])

            # save the file as a .csv
            df.to_csv('%s/%s_%s.csv'%(save_path, file_name, self.todays_week()), index=False)
            
            # check if there is already an existing file to delete
            self.older_csv_file_exist(self.DB, file_name)            

            return print(df,"csv file saved at %s with the name %s"%(save_path, file_name))
        
        else:
            return print("Key file was already downloaded this week")
        
    ############################################################################################################################
    # 3. 
    # define function, to return all possible submissions of a firm
    def return_submissions(self, ticker, file_name='submissions'):
        # create a subfolder for the ticker
        if not os.path.exists('%s/%s'%(self.DB, ticker)):
            os.makedirs('%s/%s'%(self.DB, ticker))
            
        # check if it already exists
        if self.csv_file_exist('%s/%s'%(self.DB, ticker), file_name) != True:            
            # get the CIK for the ticker   
            key_file = pd.read_csv('%s/%s'%(self.DB, 'EdgarCompanies_%s.csv'%(self.todays_week())))      
            cik = str(key_file.loc[key_file['ticker']==ticker, 'cik'].values[0])

            # the cik is always used with 10 digits. Therefore zeros are missing
            missing_zero = (10 - len(cik)) * '0'        
            complete_cik = missing_zero + cik

            # open recent submission .json file        
            URL = 'https://data.sec.gov/submissions/CIK%s.json'%(complete_cik)
            # request and save data to JSON
            r = requests.get(URL, headers=self.hdr)
            data = r.json()        
            submissions = pd.DataFrame(data['filings']['recent'])

            for add_sub in data['filings']['files']:
                add_json = add_sub['name']

                time.sleep(uniform(1, 2)) # break between 1 - 2 seconds     

                # open recent submission .json file        
                URL = 'https://data.sec.gov/submissions/%s'%(add_json)
                # request and save data to JSON
                r = requests.get(URL, headers=self.hdr)
                add_data = r.json()                  
                # add the additional submissons
                submissions = pd.concat([submissions, pd.DataFrame(add_data)], ignore_index=True)

            # save the submissions
            submissions.sort_values('filingDate', inplace=True, ascending=False)        
            submissions.to_csv("%s/%s\submissions_%s.csv"%(self.DB, ticker, self.todays_week()))

            # check if there is already an existing file to delete
            self.older_csv_file_exist('%s/%s'%(self.DB, ticker), file_name) 
        else:
            return print("Submission file was already downloaded this week")
        
    ############################################################################################################################
    # 4. 
    # Download all XBRL fillings of a firm as a .zip and extract the files
    def download_zip_submission(self, ticker, form, start_year=2010, subm_df_name='submissions'):
        # get the cik for the ticker
        key_file = pd.read_csv('%s/%s'%(self.DB, 'EdgarCompanies_%s.csv'%(self.todays_week())))      
        cik = str(key_file.loc[key_file['ticker']==ticker, 'cik'].values[0])
        
        # open the saved submission df
        for fname in os.listdir('%s/%s'%(self.DB, ticker)):
            if fname.startswith('%s'%(subm_df_name)) == True:
                name_sub_df = fname
        submissions = pd.read_csv('%s/%s/%s'%(self.DB, ticker, name_sub_df))   
        submissions['reportDate'] =  pd.to_datetime(submissions['reportDate'], format='%Y-%m-%d')
        # get all the relevant accession numbers, which are also available as an XBRL document
        submissions = submissions.loc[((submissions['reportDate'].dt.year >= start_year) &
                                       (submissions['form'] == form)&(submissions['isXBRL'] == 1))]        
        acc_numbers = submissions['accessionNumber'].to_list()      
        
        # folder names
        # list of all content in a directory, filtered so only directories are returned
        # from: https://stackoverflow.com/questions/29206384/python-folder-names-in-the-directory
        path = '%s/%s/'%(self.DB, ticker)
        dir_list = [directory for directory in os.listdir(path) if os.path.isdir(path+directory)]
        
        # download only the files, which weren't already downloaded
        acc_numbers = [a for a in acc_numbers if a not in dir_list]
        
        # if the list is not empty
        if acc_numbers:        
            for number in acc_numbers:
                # remove the dash of the number
                no_dash_num = re.sub('-', '', number)

                # URL of the .ZIP of the filing
                url = 'https://www.sec.gov/Archives/edgar/data/%s/%s/%s-xbrl.zip'%(cik, no_dash_num, number)

                # destination
                # create the directory if it doesn't exist  
                dst = '%s/%s/ZipDummy'%(self.DB, ticker)
                if not os.path.exists(dst):
                    os.makedirs(dst)

                extraction_folder = '%s/%s/%s'%(self.DB, ticker, number)
                # folder to which to extract the files of zip file
                if not os.path.exists(extraction_folder):
                    os.makedirs(extraction_folder)

                time.sleep(uniform(1, 2)) # break between 1 - 2 seconds  
                # specify downloader
                opener = urllib.request.build_opener()
                opener.addheaders = self.hdr.items()
                urllib.request.install_opener(opener)
                urllib.request.urlretrieve(url, '%s\%s.zip'%(dst, number))

                # extract the zip file
                with zipfile.ZipFile('%s/%s.zip'%(dst, number), 'r') as zip_ref:
                    zip_ref.extractall('%s'%(extraction_folder))

                # delete the zip file after extraction
                shutil.rmtree(dst)        

In [4]:
firm_ticker = 'MSFT'
filling_form = '10-K'
################################################################################################################################
# 1
# initiate the class
SEC = SEC_DB()

# insert directory and user agent
SEC.init(directory,
         header={'User-Agent': user_agent})
# 2
SEC.edgar_companies()
# 3
SEC.return_submissions(ticker=firm_ticker)
# 4
SEC.download_zip_submission(ticker=firm_ticker, form=filling_form, start_year=2010, subm_df_name='submissions')